# Draft `pyGIMLi(emg3d)`

**NEEDS**
- `pyGIMLi`
- `emg3d`
- `discretize`
- `xarray`
- `h5py`

**Current Limitations**
- Only isotropic models supported, without el. perm. and magn. perm.

An attempt at using `pyGIMLi` as an inversion framework for `emg3d` computations.

For developping purposes, we take a very simple grid/model/survey:
- Coarse mesh, no stretching (potentially too small).
- Simple double-halfspace model water-subsurface with a resistive block.
- Survey: A single 2D line, 6 sources, 1 frequency.

=> For this dev-implementation we also do inversion crime, using the same grid for forward modelling and inversion.

In [ ]:
import emg3d
import numpy as np
import pygimli as pg
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, SymLogNorm

In [ ]:
%matplotlib notebook

## Load survey (incl. data) and initial model

In [ ]:
data = emg3d.load('pginv.h5')
survey = data['survey']
model = data['model']
grid = model.grid

## Jacobian Wrapper

In [ ]:
class EMG3DJacobian(pg.Matrix):
    
    def __init__(self, sim):
        super().__init__()
        self.sim = sim

    def cols(self):
        # sim.model.size corresponds to the number of cells
        return self.sim.model.size

    def rows(self):
        # sim.survey.count corresponds to the number of non-NaN data points.
        return self.sim.survey.count

    def mult(self, x):
        """J * x """
        # reshape or ravel
        jvec = self.sim.jvec(x.reshape(self.sim.model.shape, order='F'))
        data = jvec[self.sim._finite_data]
        return np.hstack((data.real, data.imag))

    def transMult(self, x):
        """J.T * x = (x * J.T)^T """
        data = np.ones(survey.data.observed.shape, dtype=self.sim.data.observed.dtype)*np.nan
        data[self.sim._finite_data] = x[:x.size//2] + 1j*x[x.size//2:]
        return self.sim.jtvec(data).ravel('F')

## Forward Wrapper

In [ ]:
class MyFWD(pg.Modelling):
    
    def __init__(self, sim):
        
        # Should it be here or at the end of the __init__?
        super().__init__()
        
        # Store the simulation
        # (I replaced «fop» by «sim».)
        self.sim = sim
        
        # Translate discretize TensorMesh to pg-Grid
        self.mesh = pg.createGrid(
            x=sim.model.grid.nodes_x,
            y=sim.model.grid.nodes_y,
            z=sim.model.grid.nodes_z,
        )
        
        self.J = EMG3DJacobian(sim)
        self.setJacobian(self.J)

    def response(self, model):
        # do a lot of checks and cleanups
        
        # Clean emg3d-simulation, so things are recomputed
        self.sim.clean('computed')
        
        # Replace model
        self.sim.model = emg3d.Model(self.sim.model.grid, property_x=model)
        
        # Compute responses for new model
        self.sim.compute()
        
        # Return the responses
        data = self.sim.data.synthetic.data[self.sim._finite_data]
        return np.hstack((data.real, data.imag))
    
    def createStartModel(self, dataVals):
        return np.ones(self.sim.model.size)
    
    def createJacobian(self, model):
        pass  # do nothing

## Run an inversion

In [ ]:
# Create an emg3d Simulation instance
sim = emg3d.simulations.Simulation(
    survey=survey,
    model=model,
    gridding='same',  # I will like to make that more flexible in the future
    max_workers=6,    # Adjust as needed
    receiver_interpolation='linear',  # Currently necessary for the gradient
    # solver_opts,
    tqdm_opts=False,  # Switch off verbose progress bars
)

# Not sure if this is the best way, but for now it works
sim._finite_data = np.isfinite(sim.data.observed.data)

In [ ]:
fop = MyFWD(sim)
INV = pg.Inversion(fop=fop, verbose=True)

INV.transData = pg.trans.TransSymLog(survey.noise_floor)
INV.transModel = pg.trans.TransLogLU(1, 1000)

# Regularization: Setting active/passive cells would be great,
# e.g., de-activating air and water for the inversion.
# And of course other regularizations (smoothness etc).
# INV.setRegularization(limits=[], correlationLengths=[...])


data = sim.data.observed.data[sim._finite_data].copy()
model = INV.run(
    dataVals=data,
    verbose=True,
    # emg3d would have the standard deviation, existing of
    # relative and absolute error. Could that be provided?
    errorVals=np.ones(data.size)*sim.survey.relative_error, 
)

In [ ]:
emg3d.Report(['pygimli', 'pgcore'])